In [199]:
import pandas as pd
from nltk import word_tokenize
import re
import nltk
import numpy as np
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.svm import LinearSVC
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,f1_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


**Preprocessing Data**

In [200]:
df=pd.read_csv("p2_train.csv")
df_test=pd.read_csv("p2_test.csv")
df_question=df["question"]
df_response=df["response"]
df_question_test=df_test["question"]
df_response_test=df_test["response"]


In [201]:
def preprocessor(text):
  review=re.sub('\[[^]]*\]', ' ', str(text))
  review=re.sub('[^a-zA-z]', ' ', str(text))
  review=review.lower().split()
  return review
df_question=df_question.apply(preprocessor)
df_response=df_response.apply(preprocessor)
df_question_test=df_question_test.apply(preprocessor)
df_response_test=df_response_test.apply(preprocessor)

**POS Tagging(Spacy)**

In [206]:
lst=[]
for index, row in df_question.to_frame().iterrows():
  for i in row:
    string=" "
    string=string.join(i)
    tokens=word_tokenize(string)
    lst.append(nltk.pos_tag(tokens))
lst1=[]
for index, row in df_response.to_frame().iterrows():
  for i in row:
    string=" "
    string=string.join(i)
    tokens=word_tokenize(string)
    lst1.append(nltk.pos_tag(tokens))


In [207]:
lst_test=[]
for index, row in df_question_test.to_frame().iterrows():
  for i in row:
    string=" "
    string=string.join(i)
    tokens=word_tokenize(string)
    lst_test.append(nltk.pos_tag(tokens))
lst1_test=[]
for index, row in df_response_test.to_frame().iterrows():
  for i in row:
    string=" "
    string=string.join(i)
    tokens=word_tokenize(string)
    lst1_test.append(nltk.pos_tag(tokens))



In [208]:
res=[]
for i in lst:
  combined_list=[]
  for j in i:
    combined_list.append(j[0]+"/"+j[1])
  res.append(combined_list)
res1=[]
for i in lst1:
  combined_list=[]
  for j in i:
    combined_list.append(j[0]+"/"+j[1])
  res1.append(combined_list)
   

In [209]:
res_test=[]
for i in lst_test:
  combined_list=[]
  for j in i:
    combined_list.append(j[0]+"/"+j[1])
  res_test.append(combined_list)
res1_test=[]
for i in lst1_test:
  combined_list=[]
  for j in i:
    combined_list.append(j[0]+"/"+j[1])
  res1_test.append(combined_list)


In [210]:
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [211]:
import spacy
import en_core_web_lg
nlp=en_core_web_lg.load()

**Calculating vectors of word**

In [212]:
def get_vector(x):
    doc=nlp(x)
    vec=doc.vector
    return vec

temp1,temp2=[],[]
for i in res:
    string=" "
    string=string.join(i)
    temp1.append(string)
for i in res1:
    string=" "
    string=string.join(i)
    temp2.append(string)
question,response=pd.Series(temp1),pd.Series(temp2)
question=question.apply(lambda x: get_vector(x))
response=response.apply(lambda x: get_vector(x))
question,response=question.to_numpy(),response.to_numpy()
question,response=question.reshape(-1,1),response.reshape(-1,1)
question=np.concatenate(np.concatenate(question,axis=0),axis=0).reshape(-1,300)
response=np.concatenate(np.concatenate(response,axis=0),axis=0).reshape(-1,300)


In [213]:
final_vector=np.concatenate((question,response),axis=1)
final_vector.shape

(1640, 600)

In [214]:
temp1,temp2=[],[]
for i in res_test:
    string=" "
    string=string.join(i)
    temp1.append(string)
for i in res1_test:
    string=" "
    string=string.join(i)
    temp2.append(string)
question,response=pd.Series(temp1),pd.Series(temp2)
question=question.apply(lambda x: get_vector(x))
response=response.apply(lambda x: get_vector(x))
question,response=question.to_numpy(),response.to_numpy()
question,response=question.reshape(-1,1),response.reshape(-1,1)
question=np.concatenate(np.concatenate(question,axis=0),axis=0).reshape(-1,300)
response=np.concatenate(np.concatenate(response,axis=0),axis=0).reshape(-1,300)

In [215]:
final_vector_test=np.concatenate((question,response),axis=1)
final_vector_test.shape


(410, 600)

In [216]:
X_train,X_test,y_train,y_test=final_vector,final_vector_test,df["type"],df_test["type"]

In [217]:
clf = RandomForestClassifier(n_estimators=15, random_state=10,class_weight="balanced")
clf.fit(X_train,y_train)
predictions=clf.predict(X_test)
print("Accuracy is",accuracy_score(predictions,y_test))
print("F1 Score is",f1_score(predictions,y_test,average='weighted'))
print("Precision Score is",precision_score(predictions,y_test,average="weighted"))
print("Recall Score is",recall_score(predictions,y_test,average="weighted"))


Accuracy is 0.7682926829268293
F1 Score is 0.8424488544566213
Precision Score is 0.9408021872222772
Recall Score is 0.7682926829268293


In [218]:
print(confusion_matrix(predictions,y_test, labels=["answered", "attacked", "irrelevant","agreed"]))

[[311  38  35  12]
 [  3   0   0   0]
 [  6   1   3   0]
 [  0   0   0   1]]


**Doc2Vec(POS tagging)**

In [219]:
tagged_question = [TaggedDocument(d, [i]) for i, d in enumerate(res)]
tagged_response = [TaggedDocument(d, [i]) for i, d in enumerate(res1)]

In [220]:
model_question= Doc2Vec(vector_size=150)
model_question.build_vocab(tagged_question)
model_question.train(tagged_question, epochs=model_question.iter, total_examples=model_question.corpus_count)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [221]:
array=np.zeros((len(res),150))
for i in range(len(df_question)):
  array[i,:]=model_question.docvecs[i].reshape((1,150))
doc=pd.DataFrame(array)


In [222]:
model_response=Doc2Vec(vector_size=150)
model_response.build_vocab(tagged_response)
model_response.train(tagged_response, epochs=model_response.iter, total_examples=model_response.corpus_count)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [223]:
array=np.zeros((len(res1),150))
for i in range(len(df_response)):
  array[i,:]=model_response.docvecs[i].reshape((1,150))
doc1=pd.DataFrame(array)


In [224]:
fv=np.concatenate((doc,doc1),axis=1)
fv.shape

(1640, 300)

In [225]:
tagged_question_test = [TaggedDocument(d, [i]) for i, d in enumerate(res_test)]
tagged_response_test = [TaggedDocument(d, [i]) for i, d in enumerate(res1_test)]

In [226]:
model_question_test= Doc2Vec(vector_size=150)
model_question_test.build_vocab(tagged_question_test)
model_question_test.train(tagged_question_test, epochs=model_question_test.iter, total_examples=model_question_test.corpus_count)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [227]:
array=np.zeros((len(res_test),150))
for i in range(len(df_question_test)):
  array[i,:]=model_question_test.docvecs[i].reshape((1,150))
doc=pd.DataFrame(array)


In [228]:
model_response_test=Doc2Vec(vector_size=150)
model_response_test.build_vocab(tagged_response_test)
model_response_test.train(tagged_response_test, epochs=model_response_test.iter, total_examples=model_response_test.corpus_count)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [229]:
array=np.zeros((len(res1_test),150))
for i in range(len(df_response_test)):
  array[i,:]=model_response_test.docvecs[i].reshape((1,150))
doc1=pd.DataFrame(array)


In [230]:
fv_test=np.concatenate((doc,doc1),axis=1)
fv_test.shape

(410, 300)

In [231]:
X_train,X_test,y_train,y_test=final_vector,final_vector_test,df["type"],df_test["type"]

In [232]:
clf = RandomForestClassifier(n_estimators=10, random_state=5,class_weight="balanced")
clf.fit(X_train,y_train)
predictions=clf.predict(X_test)
print("Accuracy is",accuracy_score(predictions,y_test))
print("F1 Score is",f1_score(predictions,y_test,average='weighted'))
print("Precision Score is",precision_score(predictions,y_test,average="weighted"))
print("Recall Score is",recall_score(predictions,y_test,average="weighted"))


Accuracy is 0.7195121951219512
F1 Score is 0.7593158583146663
Precision Score is 0.8097363483756295
Recall Score is 0.7195121951219512


In [233]:
print(confusion_matrix(predictions,y_test, labels=["answered", "attacked", "irrelevant","agreed"]))

[[288  35  32  10]
 [ 21   3   3   2]
 [ 11   1   3   0]
 [  0   0   0   1]]


**Sentence Embeddings(Spacy)**

In [234]:
que=[]
for i in df_question:
  que.append(i)
resp=[]
for i in df_response:
  resp.append(i)
que_test=[]
for i in df_question_test:
  que_test.append(i)
resp_test=[]
for i in df_response_test:
  resp_test.append(i)

 
  

In [235]:
def get_vector(x):
    doc=nlp(x)
    vec=doc.vector
    return vec
temp1,temp2=[],[]
for i in que:
    string=" "
    string=string.join(i)
    temp1.append(string)
for i in resp:
    string=" "
    string=string.join(i)
    temp2.append(string)
question,response=pd.Series(temp1),pd.Series(temp2)
question=question.apply(lambda x: get_vector(x))
response=response.apply(lambda x: get_vector(x))
question,response=question.to_numpy(),response.to_numpy()
question,response=question.reshape(-1,1),response.reshape(-1,1)
question_train=np.concatenate(np.concatenate(question,axis=0),axis=0).reshape(-1,300)
response_train=np.concatenate(np.concatenate(response,axis=0),axis=0).reshape(-1,300)

In [236]:
final_vector_1=np.concatenate((question_train,response_train),axis=1)
final_vector_1.shape

(1640, 600)

In [237]:
temp1,temp2=[],[]
for i in que_test:
    string=" "
    string=string.join(i)
    temp1.append(string)
for i in resp_test:
    string=" "
    string=string.join(i)
    temp2.append(string)
question,response=pd.Series(temp1),pd.Series(temp2)
question=question.apply(lambda x: get_vector(x))
response=response.apply(lambda x: get_vector(x))
question,response=question.to_numpy(),response.to_numpy()
question,response=question.reshape(-1,1),response.reshape(-1,1)
question_test=np.concatenate(np.concatenate(question,axis=0),axis=0).reshape(-1,300)
response_test=np.concatenate(np.concatenate(response,axis=0),axis=0).reshape(-1,300)

In [238]:
final_vector_1_test=np.concatenate((question_test,response_test),axis=1)
final_vector_1_test.shape

(410, 600)

In [239]:
X_train,X_test,y_train,y_test=final_vector,final_vector_test,df["type"],df_test["type"]

In [240]:
clf=LinearSVC(class_weight='balanced')
clf.fit(X_train,y_train)
predictions=clf.predict(X_test)
print("Accuracy is",accuracy_score(predictions,y_test))
print("F1 Score is",f1_score(predictions,y_test,average='weighted'))
print("Precision Score is",precision_score(predictions,y_test,average="weighted"))
print("Recall Score is",recall_score(predictions,y_test,average="weighted"))


Accuracy is 0.6292682926829268
F1 Score is 0.5958796466007044
Precision Score is 0.5761118791349857
Recall Score is 0.6292682926829268


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [241]:
print(confusion_matrix(predictions,y_test, labels=["answered", "attacked", "irrelevant","agreed"]))

[[233  16  15   8]
 [ 36  12   9   1]
 [ 37   9  11   2]
 [ 14   2   3   2]]


In [242]:
clf = RandomForestClassifier(n_estimators=10, random_state=0,class_weight="balanced")
clf.fit(X_train,y_train)
predictions=clf.predict(X_test)
print("Accuracy is",accuracy_score(predictions,y_test))
print("F1 Score is",f1_score(predictions,y_test,average='weighted'))
print("Precision Score is",precision_score(predictions,y_test,average="weighted"))
print("Recall Score is",recall_score(predictions,y_test,average="weighted"))


Accuracy is 0.748780487804878
F1 Score is 0.8125994521977304
Precision Score is 0.8922593068035943
Recall Score is 0.748780487804878


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [243]:
print(confusion_matrix(predictions,y_test, labels=["answered", "attacked", "irrelevant","agreed"]))

[[304  36  33  11]
 [ 10   0   2   1]
 [  6   3   3   1]
 [  0   0   0   0]]


**PCA(dimensionality reduction)**

In [244]:
pca_train=np.concatenate((final_vector,final_vector_1),axis=1)
pca_train.shape


(1640, 1200)

In [245]:
pca_test=np.concatenate((final_vector_test,final_vector_1_test),axis=1)
pca_test.shape

(410, 1200)

In [246]:
scaler = StandardScaler()
pca_train = scaler.fit_transform(pca_train)
pca_test =scaler.fit_transform(pca_test)

In [247]:
pca = PCA(n_components=100)
pca.fit(pca_train)
a=pca.transform(pca_train)

In [248]:
pca = PCA(n_components=100)
pca.fit(pca_test)
b=pca.transform(pca_test)

**Set 1: Baseline Features(Evaluation Metrics)**

In [249]:
X_train,X_test,y_train,y_test=a,b,df["type"],df_test["type"]

In [256]:
clf=LinearSVC()
clf.fit(X_train,y_train)
predictions=clf.predict(X_test)
print("Accuracy is",accuracy_score(predictions,y_test))
print("F1 Score is",f1_score(predictions,y_test,average='weighted'))
print("Precision Score is",precision_score(predictions,y_test,average="weighted"))
print("Recall Score is",recall_score(predictions,y_test,average="weighted"))


Accuracy is 0.6121951219512195
F1 Score is 0.6063913888667675
Precision Score is 0.6021019921990718
Recall Score is 0.6121951219512196


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [257]:
print(confusion_matrix(y_test,predictions, labels=["answered", "attacked", "irrelevant","agreed"]))

[[241  45  31   3]
 [ 36   3   0   0]
 [ 26   6   6   0]
 [ 11   1   0   1]]


In [258]:
clf = RandomForestClassifier(n_estimators=10, random_state=20,class_weight="balanced")
clf.fit(X_train,y_train)
predictions=clf.predict(X_test)
print("Accuracy is",accuracy_score(predictions,y_test))
print("F1 Score is",f1_score(predictions,y_test,average='weighted'))
print("Precision Score is",precision_score(predictions,y_test,average="weighted"))
print("Recall Score is",recall_score(predictions,y_test,average="weighted"))


Accuracy is 0.748780487804878
F1 Score is 0.8115681304824148
Precision Score is 0.8920418497580725
Recall Score is 0.748780487804878


In [259]:
print(confusion_matrix(y_test, predictions,labels=["answered", "attacked", "irrelevant","agreed"]))

[[303  10   5   2]
 [ 36   2   1   0]
 [ 33   3   2   0]
 [ 13   0   0   0]]


In [262]:
df_final = pd.read_csv('p2_test.csv')
df_final['predicted_type']=predictions
df_final=df_final.rename(columns={"label": "gold_type"})
df_final

,thread_id,question_id,response_id,no_turn_q_id,quoted_q_id,precedent,question,subsequent,response,type,predicted_type
0,232fcl,232fcl,cgsrxd3,NaN,q_27319,NaN,Now if Julie was underage (let's say you and ...,NaN,"\nNo. She's happy, he's happy, who am I to end...",irrelevant,answered
1,221bir,cgikp7a,cginbgy,n_228942,q_28801,"And Egypt...there was a lot of chaos, a lot of...",Did the protests motivate the military to act?,Undoubtedly. But it was still the military tha...,Seems a weird example to use in defence of an ...,attacked,answered
2,20s1x1,cg6ay9r,cg6f18i,n_244708,q_30570,&gt; Hello! Both my major AND my minor are due...,Can you illustrate how a single class in CS w...,Your statement that way more happiness is ach...,"Oh gods yes. First of all, most students aren'...",answered,answered
3,1zyf3k,cfzfpej,cfzg0hy,n_254761,q_31795,"Oh, come on, now you're just trying to misunde...","Didn't I even say, that I am completely and 10...",Let me make it very clear: I am an universal ...,"Sure, but you still obviously have some precon...",irrelevant,answered
4,1yfntu,cfka1pw,cfkar3s,NaN,q_34113,NaN,However if we forget that debate for a minute...,NaN,I'd be willing to guess that this question is ...,attacked,answered
...,...,...,...,...,...,...,...,...,...,...,...
405,2sr2st,cns3qwy,cns4t4s,n_632548,q_78351,"So, just to be clear, if a woman refused to ha...",Am I getting this right?,"But if it's just about the money, would you b...",Yes,answered,answered
406,38x7jw,38x7jw,crykb9x,NaN,q_8321,NaN,Further evidence that merely being overweight ...,NaN,\n,irrelevant,irrelevant
407,2ija3a,cl2zs70,cl33t6t,n_65173,q_8864,Why? Just because it's expected of us?,By who?,This is hardly ever a good reason for doing or...,Maybe I didn't phrase this first point well. B...,answered,answered
408,2hqis0,ckve18f,ckvh0bt,n_70808,q_9521,&gt; You don't have to but you should be able ...,Why are the consequences of everyone taking a...,If everyone became a dental hygienist we'd al...,It's worth considering the societal implicatio...,answered,answered


In [264]:
df_final.to_csv('test_output_baselinesolution.csv')
